**Import Libraries**

In [50]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

**Drive** **Connection**

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Data Load From Drive**

In [52]:
train_Normal = np.load("/content/drive/MyDrive/SoftCom/train_Normal_128.npy",allow_pickle=False, fix_imports=True)
train_virus = np.load("/content/drive/MyDrive/SoftCom/train_Virus_128.npy",allow_pickle=False, fix_imports=True)
test_Normal = np.load("/content/drive/MyDrive/SoftCom/test_Normal_128.npy",allow_pickle=False, fix_imports=True)
test_virus = np.load("/content/drive/MyDrive/SoftCom/test_Virus_128.npy",allow_pickle=False, fix_imports=True)

**Data Preprocessing**

In [53]:
#Train Data
train_normal = torch.from_numpy(train_Normal)
train_virus = torch.from_numpy(train_virus)
train_normal_label = torch.zeros(len(train_normal))
train_virus_label = torch.ones(len(train_virus))
train_data = torch.cat((train_normal, train_virus), axis=0)
train_label = torch.cat((train_normal_label, train_virus_label), axis=0)

train_data = torch.reshape(train_data, (2686,1,128,128))


#Test Data
test_normal = test_Normal.astype(np.float32)#64
test_normal = torch.from_numpy(test_normal)
test_virus = torch.from_numpy(test_virus)
test_normal_label = torch.zeros(len(test_normal))
test_virus_label = torch.ones(len(test_virus))
test_data = torch.cat((test_normal, test_virus), axis=0)
test_label = torch.cat((test_normal_label, test_virus_label), axis=0)
print(test_data.shape)
test_data = torch.reshape(test_data,(382,1,128,128))

torch.Size([382, 128, 128, 1])


**Seeding**

In [54]:
torch.manual_seed(101)


In [55]:
x=int(torch.rand(1,)*1000)

**Slicing**

In [56]:
traindata = [train_data[i] for i in range(len(train_data))]
train = torch.stack([d[0] for d in traindata], dim=0)
train=train[101:x]        


testdata = [test_data[i] for i in range(len(test_data))]
test = torch.stack([d[0] for d in testdata], dim=0)       
test=test[0:382]

In [57]:
train_y = train_label  
test_y = test_label


**Loading Train and Test Data**

In [58]:
train_loader = torch.utils.data.DataLoader(dataset= (train,train_y) , 
                                           shuffle=True)   # It's better to shuffle the whole training dataset! 

test_loader = torch.utils.data.DataLoader(dataset= (test,test_y), 
                                         shuffle=False)

**CNN Model Architeture**

In [59]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=1, out_channels=16,kernel_size=5, stride=1, padding=2)
        self.cnn_layer_2 = nn.Conv2d(in_channels=16, out_channels=32,kernel_size=5, stride=1, padding=2)
        
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(2,2)
        
        self.linear_layer_1 = nn.Linear(32*32*32, 512) 
        self.linear_layer_2 = nn.Linear(512, 128)
        self.linear_layer_3 = nn.Linear(128, 2)   
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(.2)
        
        # self.flatten = nn.Flatten()
        
    def forward(self, x):

        x = self.cnn_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x) 
        
        #print(x.shape)
        
        x = self.cnn_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #print(x.shape)
        
        x = self.flatten(x)# ekhon koto gula pixel oitar dimention calculate 
        #print(x.shape)
        
        x = self.linear_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_3(x)
        #logits = self.sigmoid(x)
        return x

**Model Creation**

In [60]:
model = NeuralNetwork(128*128)
print(model)

NeuralNetwork(
  (cnn_layer_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_layer_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer_1): Linear(in_features=32768, out_features=512, bias=True)
  (linear_layer_2): Linear(in_features=512, out_features=128, bias=True)
  (linear_layer_3): Linear(in_features=128, out_features=2, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
)


In [61]:
from torchsummary import summary
summary(model,(1,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             416
           Dropout-2         [-1, 16, 128, 128]               0
              ReLU-3         [-1, 16, 128, 128]               0
         MaxPool2d-4           [-1, 16, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]          12,832
           Dropout-6           [-1, 32, 64, 64]               0
              ReLU-7           [-1, 32, 64, 64]               0
         MaxPool2d-8           [-1, 32, 32, 32]               0
           Flatten-9                [-1, 32768]               0
           Linear-10                  [-1, 512]      16,777,728
          Dropout-11                  [-1, 512]               0
             ReLU-12                  [-1, 512]               0
           Linear-13                  [-1, 128]          65,664
          Dropout-14                  [

**Optimizer**

In [62]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [63]:
def trainModel(model, loss_fn, optimizer):
    model.train()

    batch = 200

    loss = 0
    for i in range(train.shape[0]):
      x, y = torch.reshape(train[i],(1,1,128,128)), torch.tensor([train_y[i]], dtype=torch.float)
      label=torch.zeros([1,2,], dtype=torch.float32)
      label[0,int(y.item())]=1
      # Compute prediction error
      pred = model(x)     
      #print(pred)
      #print(label)
      loss += loss_fn(pred, label)
      
      if i>0 and (i+1)%batch == 0:
          # Backpropagation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          print(f'Training Loss: {loss.item():.4f}', end="\r")
          loss = 0
    print()

**Model testing**

In [64]:
def testModel(model, loss_fn):
    model.eval()

    size = test.shape[0]
    correct=0
    loss = 0
    total =382
    with torch.no_grad():
      for i in range(test.shape[0]):
        x, y = torch.reshape(test[i],(1,1,128,128)), torch.tensor([test_y[i]], dtype=torch.float)
        label=torch.zeros([1,2,], dtype=torch.float32)
        label[0,int(y.item())]=1
        pred = model(x)
        #print(pred)
        predicted = torch.argmax(pred)
        #print(predicted)
        #print(y)


        # Total correct predictions
        correct += (predicted == int(y)).sum()

        loss += loss_fn(pred, label)
      
    loss /= size
    accuracy = 100 * correct.item() / total

    # Print Loss
    print('Loss: {}. Accuracy: {}'.format({loss}, accuracy))

In [65]:
epochs = 4
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainModel(model, loss_fn, optimizer)
    testModel(model, loss_fn)
print("Done!")

Epoch 1
-------------------------------

Loss: {tensor(1.4164)}. Accuracy: 38.7434554973822
Epoch 2
-------------------------------

Loss: {tensor(1.4164)}. Accuracy: 38.7434554973822
Epoch 3
-------------------------------

Loss: {tensor(1.4164)}. Accuracy: 38.7434554973822
Epoch 4
-------------------------------

Loss: {tensor(1.4164)}. Accuracy: 38.7434554973822
Done!
